In [ ]:
from sklearn.metrics import mean_squared_error, confusion_matrix, auc

# Metrics utils

In [ ]:
def roc_params(metric, label, interp=True):
    fpr = []
    tpr = []
    thr = []
    thr_list = list(np.linspace(0, metric.max(),1001))

    fp = 1
    ind = 0
    while fp > 0:
        threshold = thr_list[ind]
        ind += 1

        y = (metric>threshold)
        tn, fp, fn, tp = confusion_matrix(label, y).ravel()

        fpr.append( fp/(tn + fp) )
        tpr.append( tp/(tp + fn) )
        thr.append( threshold )

    while tp > 0:
        threshold = thr_list[ind]
        ind += 1
        y = (metric>threshold)
        tn, fp, fn, tp = confusion_matrix(label, y).ravel()

    
    fpr = fpr[::-1]
    tpr = tpr[::-1]
    thr = thr[::-1]

    if interp:
        fpr_base = np.linspace(0, 1, 101)
        tpr = list(np.interp(fpr_base, fpr, tpr))
        thr = list(np.interp(fpr_base, fpr, thr))
        fpr = list(fpr_base)

    fpr.insert(0, 0)
    tpr.insert(0, 0)
    thr.insert(0, threshold)

    return tpr, fpr, thr

def compute_auc(tpr, fpr):
    auc = 0
    for i in range(1, len(fpr)):
        auc += (fpr[i] - fpr[i - 1]) * (tpr[i] + tpr[i - 1]) / 2
    return auc


# Synthetic graph generation

In [ ]:
def generate_synthetic_graph(N, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    N_grid_points = int(np.floor(0.9*N))
    N_rand_points = int(np.ceil(0.1*N))

    connected = False

    while not connected:

        # Generating grid points

        # Initialize starting position
        current_position = [0, 0]

        # Store visited points
        visited_points = set()
        visited_points.add(tuple(current_position))

        # Generate N points
        while len(visited_points) < N_grid_points:
            # Update the current position by taking a vertical or horizontal step
            # horizontal can be +-5, vertical can be +-15

            if np.random.rand()>0.5:
                current_position[0] += np.random.choice([-5, 5])
            else:
                current_position[1] += np.random.choice([-14, 14])

            # Add the new position to the set of visited points
            visited_points.add(tuple(current_position))

        points_list = list(visited_points)

        # Generating random points
        reference_pos = np.random.choice(a=N_grid_points, size=N_rand_points, replace=False)
        reference_points = [list(points_list[i]) for i in reference_pos]

        for point in reference_points:
            point[0] += 10*np.random.random()
            point[1] += 10*np.random.random()
            visited_points.add(tuple(point))
        

        # Convert the set of visited points to a list
        pos = np.array(list(visited_points))
        pos = pos + 0.5*np.random.randn(pos.shape[0], pos.shape[1])

        radius=15
        sigma=radius**2

        G = pygsp.graphs.NNGraph(pos,
                                NNtype='radius',
                                epsilon = radius, sigma = sigma,
                                center=False, rescale=False)
        connected = G.is_connected()

    return G

# Data generation

In [ ]:
def generate_ramp(G, size):
    # Auxiliary function for generating data.
    # Returns array with "size" random ramps on the given graph. A ramp emulates regions with different behaviors.
    # Direction of each ramp is randomly horizontal or vertical
    # Slope, start and end positions of the ramp are random, within some constraints.
    
    pos = G.coords
    min_slope = 0.1
    max_slope = 1
    
    # Initialize an empty matrix to store ramp vectors
    ramp_matrix = np.zeros((len(pos), size))

    for i in range(size):
        # Randomly choose the direction of the ramp (0, horizontal or 1, vertical)
        if np.random.rand() > 0.5:
            direction = 0
        else:
            direction = 1

        # Generate a random slope within the specified maximum slope
        slope = np.random.uniform(low=min_slope, high=max_slope)

        # Calculate the minimum ramp length as a fraction of the peak-to-peak range
        min_slope_dist = 0.25 * pos[:, direction].ptp()

        # Determine the starting and ending positions
        start = pos[:, direction].min() + pos[:, direction].ptp() * np.random.rand() * 0.5
        end = start + min_slope_dist + (pos[:, direction].max() - start - min_slope_dist) * np.random.rand()

        # Generate the ramp vector based on the slope and the position within the range
        ramp = slope * (pos[:, direction] - start)

        # Applying a mask that defines where the ramp exists
        mask = (pos[:, direction] >= start) * (pos[:, direction] < end)
        ramp = ramp * mask

        # Set values beyond the end of the ramp to the maximum ramp value
        ramp[pos[:, direction] >= end] = ramp.max()

        # Store the ramp vector in the matrix
        ramp_matrix[:, i] = ramp

    return ramp_matrix

def generate_smooth(G, size):
    # Auxiliary function for generating data
    # Returns array with "size" smooth signals by filtering white noise with the two first eigenvectors of the graph
    
    w, V = np.linalg.eigh(G.L.toarray())

    # Low-pass filter
    h = np.ones(len(w))
    h[0] = 1
    h[1] = 0.1
    h[2:] = 0 

    # Generating and filtering white noise to create a smooth graph signal
    displacement = np.random.randn(G.N, size) 
    displacement = V @ np.diag(h) @ V.T @ displacement

    # Normalizing signal: average power sum(x^2)/N = 1
    displacement = np.sqrt(G.N)*displacement/(np.linalg.norm(displacement,axis=0))

    return displacement


def pick_non_adjacent(G, n):
    # Auxiliary function for generating anomalies.
    # Picks n nodes that are not adjacent to each other.
    A = G.A.toarray()
    np.fill_diagonal(A,1)

    tries = 0

    while tries < 100000:
        tries+=1

        fail = 0
        possible_nodes= list(np.arange(G.N))
        picked_nodes = []

        for node in range(n):
            if len(possible_nodes) == 0:
                fail = 1
            else:
                pick = np.random.choice(possible_nodes)
                picked_nodes.append(pick)
                neighbors = np.where(A[pick,:])[0]
                possible_nodes = list( set(possible_nodes) - set(neighbors)  )

        if not fail:
            return picked_nodes
    
    print('Maximum tries reached. Reduce n')

def generate_anomaly(G, signal, anomaly_type=1):
    # signal is expected to be a matrix containing different signals in each column
    # each column will receive different anomalies

    if anomaly_type==1:
        anomaly_factor_min = 0.05
        anomaly_factor_max = 0.15
    elif anomaly_type==2:
        anomaly_factor_min = 0.10
        anomaly_factor_max = 0.20

    N = signal.shape[0]
    size = signal.shape[1]

    # Defining number of anomalous sensors per signal. Each column of signal has a different number of anomalies
    # This value is up to 5% the number of sensors (rounded up)
    percentage_per_signal = 0.05*np.random.rand(size) + 1e-5 # value added to avoid zero
    number_per_signal = np.ceil(percentage_per_signal*N)

    anomalous_positions = [pick_non_adjacent(G,int(n)) for n in number_per_signal]

    anomaly = np.zeros(signal.shape)
    for i in range(size):
        n = int(number_per_signal[i])
        anomalous_positions = pick_non_adjacent(G,n)

        anomaly[anomalous_positions, i] = signal[:,i].ptp()*np.random.uniform(low=anomaly_factor_min,
                                                                              high=anomaly_factor_max,
                                                                              size=n)*np.random.choice([+1,-1],size=n)
        
    label = np.zeros(signal.shape)
    label[np.where(anomaly)] = 1

    return anomaly, label

def generate_data(G, size, anomaly_type=1):    

    # Generating healthy data
    displacement = generate_smooth(G, size)
    ramp = generate_ramp(G, size)
    noise = np.sqrt(0.1)*np.random.randn(G.N,size) #noise power = 0.1, SNR = 20 dB

    signal = displacement + ramp + noise

    # Introducing anomalies
    anomaly, label = generate_anomaly(G, signal, anomaly_type)
    signal = signal + anomaly
    
    return signal, label

# Weighted spectral energy utils

In [ ]:
def compute_wse(nodes, x, radius=15, decay=5, window_norm='energy', cut=2):
    S, w, V = WGFT(nodes, x, radius, decay, window_norm)
    PSD = S**2
    w[w<cut] = 0
    return np.sum(PSD*w.reshape(-1,1), axis=0) 

def WGFT(nodes, x, radius=15, decay=5, window_norm='energy'):
    # nodes is a dataframe with easting and northing coordinates
    # x is an array of dim = (number_of_nodes, number_of_samples), with at least 1 column

    # Applies window hm to signal x, for all nodes, to obtain the power spectral density S

    # x = x/np.linalg.norm(x)
    # x = x - x.mean(axis=0)
     
    G = NNGraph(nodes, radius=radius)
    w, V = np.linalg.eigh(G.L.toarray())

    # creating shapes for single matrix processing of several samples at once
    nsamples = x.shape[1] # samples of an entire graph signal. Probably the number of timestamps
    X = np.tile(x, reps=(1, G.N)) # For each node, its samples. N times

    # Creating window
    Hm = spectral_window(w, V, decay, L=G.L.toarray(), norm=window_norm)
    Hm = np.repeat(Hm,repeats=nsamples, axis=1) # For each node, its window. nsamples times.

    HmX = Hm*X

    # Power spectral density S
    S = V.T.dot(HmX)
    return S, w, V

def spectral_window(w, V, decay, L, norm='energy' ):

    h_hat = np.exp(-w*decay) # creating spectral window
    H_hat = h_hat.reshape((-1,1)).repeat(len(w), axis=1) # repeating the window N times to create shifted versions
    Hm_hat = H_hat*(V.T) # creating spectral shifted versions by hadamard product with the m-th row of V
    Hm = V.dot(Hm_hat) # obtaining windows in the vertex domain with the IGFT

    # Original normalization by norm preserves energy of the windowing
    if norm == 'energy':
        Hm = Hm/np.linalg.norm(Hm,axis=0)


    # Normalization to unitary smoothness, makes different nodes have the same baseline.
    if norm == 'smoothness':
        # w[w<0] = 0
        # L = V.dot(np.diag(w).dot(V.T))
        # Hm = Hm/np.sqrt(( (Hm_hat**2).T.dot(w) )).reshape((1,-1))
        smoothness = np.diag(Hm.T.dot(L.dot(Hm)))
        Hm = Hm/np.sqrt(smoothness).reshape((1,-1)) # sqrt(hTLh)

    return Hm

def NNGraph(nodes, radius, plotting_params = None, subgraphs=False):
    # Nodes are connected if within <radius> m of each other. This can yield disconnected subgraphs
    # if "subgraphs" is True, we also identify those subgraphs.

    sigma = radius**2  #w = np.exp(-(dist**2)/sigma) > w = 0.36 at max radius
    G = pygsp.graphs.NNGraph(nodes[['easting','northing']].values,
                            NNtype='radius',
                            epsilon = radius, sigma = sigma,
                            center=False, rescale=False)

    # Plotting
    if plotting_params is None:
        plotting_params = {'edge_color':'darkgray', 'edge_width':1.5,'vertex_color':'black', 'vertex_size':150}
    G.plotting.update(plotting_params)

    if subgraphs:
        subgraph_labels = sp.sparse.csgraph.connected_components(G.A.toarray(), directed=False)[1]
        subgraph_labels[G.d==0] = -1
        return G, pd.factorize(subgraph_labels, sort=True)[0]
    else:
        return G

# Graph filter utils

In [ ]:
def hfilter(G, cut=2):
    L = G.L.toarray()
    w, V = np.linalg.eigh(L)
    wh = np.ones(G.N)
    wh[w<cut] = 0
    Hh = V @ np.diag(wh) @ V.T
    return Hh